In [1]:
from datasets import load_dataset, load_from_disk, Dataset
import numpy as np
import pandas as pd
from transformers import RobertaTokenizer, RobertaModel
import torch
import pickle
from torch.utils.data import TensorDataset, DataLoader

In [2]:
data_base = 'data'
dataset_name = 'cnn_dailymail'

data_path = data_base+'/'+dataset_name
sentence_emb_path = data_path+'/sentence_embs'
top_sentence_emb_path = data_path+'/top_sentence_embs'

In [3]:
# dataset = load_from_disk(
#     'data/cnn_dailymail/sentences/test/test_rewards'
# )

In [4]:
# dataset = load_from_disk(
#     'data/cnn_dailymail/sentences/validation/val_rewards'
# )

In [ ]:
dataset = load_from_disk(
    'data/cnn_dailymail/sentences/train/train_rewards'
)

In [22]:
# dataset

In [33]:
splits = ['train','test','validation']
a_id_len = dict([
    (s,dict()) for s in splits
])

for split in splits:
    if split in ['test']:
        continue
    else:
        cum_sum = 0
        for a_id, a_s_idx_list in zip(dataset['article_id']['article_id'], dataset['top_sentences_index']['top_sentences_index']):
            a_len = len(a_s_idx_list)
            a_id_len[split][a_id] = (a_len, cum_sum)
            cum_sum += a_len
with open(top_sentence_emb_path+'/articleID_sentences_info.pkl', 'wb') as pickle_file:
    pickle.dump(a_id_len, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
# splits = ['train','test','validation']

# for split in splits:
#     if split in ['test', 'train']:
#         continue
#     else:
#         cum_sum = 0
#         for a_id, a_s_idx_list in zip(dataset['id'], dataset['article_top_sent']):
#             a_len = len(a_s_idx_list)
#             a_id_len[split][a_id] = (a_len, cum_sum)
#             cum_sum += a_len
# with open(top_sentence_emb_path+'/articleID_sentences_info.pkl', 'wb') as pickle_file:
#     pickle.dump(a_id_len, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
# splits = ['train','test','validation']

# for split in splits:
#     if split in ['validation', 'train']:
#         continue
#     else:
#         cum_sum = 0
#         for a_id, a_s_idx_list in zip(dataset['id'], dataset['article_top_sent']):
#             a_len = len(a_s_idx_list)
#             a_id_len[split][a_id] = (a_len, cum_sum)
#             cum_sum += a_len
# with open(top_sentence_emb_path+'/articleID_sentences_info.pkl', 'wb') as pickle_file:
#     pickle.dump(a_id_len, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)

In [4]:
with open(top_sentence_emb_path+'/articleID_sentences_info.pkl', "rb") as input_f:
    a_id_len = pickle.load(input_f)
def get_chunk_sent_idx(split, a_id, idx):
    num_s_per_chunk = 600000
    s_idx = a_id_len[split][a_id][1]+idx
    chunk_num = int(s_idx/num_s_per_chunk)+1
    chunk_s_idx = int((s_idx%num_s_per_chunk))
    return chunk_num, chunk_s_idx

In [61]:
splits = ['train','test','validation']
required_items = dict([
    (s,[]) for s in splits
])
for split in splits:
    if split in ['test','validation']:
        continue
    for a_id, a_s_idx_list in zip(dataset['article_id']['article_id'], dataset['top_sentences_index']['top_sentences_index']):
        article_items = []
        for idx, a_s_idx in enumerate(a_s_idx_list):
            chunk_num, s_idx = get_chunk_sent_idx('train', a_id, idx)
            article_items.append((chunk_num, s_idx))
        required_items[split].append((a_id, len(a_s_idx_list), article_items))

In [13]:
# splits = ['train','test','validation']

# for split in splits:
#     if split in ['test','train']:
#         continue
#     for a_id, a_s_idx_list in zip(dataset['id'], dataset['article_top_sent']):
#         article_items = []
#         for idx, a_s_idx in enumerate(a_s_idx_list):
#             chunk_num, s_idx = get_chunk_sent_idx('validation', a_id, idx)
#             article_items.append((chunk_num, s_idx))
#         required_items[split].append((a_id, len(a_s_idx_list), article_items))

In [9]:
# splits = ['train','test','validation']

# for split in splits:
#     if split in ['validation','train']:
#         continue
#     for a_id, a_s_idx_list in zip(dataset['id'], dataset['article_top_sent']):
#         article_items = []
#         for idx, a_s_idx in enumerate(a_s_idx_list):
#             chunk_num, s_idx = get_chunk_sent_idx('test', a_id, idx)
#             article_items.append((chunk_num, s_idx))
#         required_items[split].append((a_id, len(a_s_idx_list), article_items))

In [13]:
with open(top_sentence_emb_path+'/articleID_len_sents_chunk_info.pkl', 'wb') as pickle_file:
    pickle.dump(required_items, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
len(required_items['train']), required_items['train'][-1]

(287113,
 ('ffffd563a96104f5cf4493cfa701a65f31b06abf',
  10,
  [(5, 460792),
   (5, 460793),
   (5, 460794),
   (5, 460795),
   (5, 460796),
   (5, 460797),
   (5, 460798),
   (5, 460799),
   (5, 460800),
   (5, 460801)]))

In [8]:
with open(top_sentence_emb_path+'/articleID_len_sents_chunk_info.pkl', "rb") as input_f:
    required_items = pickle.load(input_f)
def get_article_sent_emb(split, chunk_num, s_idx, sent_embs_chunk=None):
    if sent_embs_chunk==None:
        with open(
          top_sentence_emb_path+'/'+split+'/'+split+'_top_sentences_part_'+str(chunk_num)+".pkl", 
          "rb"
        ) as input_f:
            sent_embs_chunk = pickle.load(input_f)
    return sent_embs_chunk[s_idx], sent_embs_chunk

In [14]:
loaded_current_chunk = None
sent_embs_chunk = None
res = []

In [ ]:
keep_articles = []
for i,(a_id, a_len, chunk_sents) in enumerate(required_items['train']):
    if i<=(244139-1):
        continue
    else:
        if len(keep_articles)==60000:
            break
        if a_len!=10:
            continue
        keep_articles.append(a_id)
        article_res = []
        for (chunk_num, s_idx) in chunk_sents:
            if loaded_current_chunk==chunk_num:
                s_emb, sent_embs_chunk = get_article_sent_emb(
                    'train', chunk_num, s_idx, sent_embs_chunk
                )
            else:
                print('  Loading chunk:', chunk_num)
                loaded_current_chunk = chunk_num
                s_emb, sent_embs_chunk = get_article_sent_emb(
                    'train', chunk_num, s_idx, None
                )
            article_res.append(s_emb)
        res.append(article_res)
        if i%500==0:
            print('Loaded',i+1,'articles of',len(required_items['train']))

In [19]:
# keep_articles = []
# for i,(a_id, a_len, chunk_sents) in enumerate(required_items['validation']):
# #     if i<=(244139-1):
# #         continue
# #     else:
#         if len(keep_articles)==60000:
#             break
#         if a_len!=10:
#             continue
#         keep_articles.append(a_id)
#         article_res = []
#         for (chunk_num, s_idx) in chunk_sents:
#             if loaded_current_chunk==chunk_num:
#                 s_emb, sent_embs_chunk = get_article_sent_emb(
#                     'validation', chunk_num, s_idx, sent_embs_chunk
#                 )
#             else:
#                 print('  Loading chunk:', chunk_num)
#                 loaded_current_chunk = chunk_num
#                 s_emb, sent_embs_chunk = get_article_sent_emb(
#                     'validation', chunk_num, s_idx, None
#                 )
#             article_res.append(s_emb)
#         res.append(article_res)
#         if i%500==0:
#             print('Loaded',i+1,'articles of',len(required_items['validation']))

  Loading chunk: 1
Loaded 1 articles of 13368
Loaded 501 articles of 13368
Loaded 1001 articles of 13368
Loaded 1501 articles of 13368
Loaded 2001 articles of 13368
Loaded 2501 articles of 13368
Loaded 3001 articles of 13368
Loaded 3501 articles of 13368
Loaded 4001 articles of 13368
Loaded 4501 articles of 13368
Loaded 5001 articles of 13368
Loaded 5501 articles of 13368
Loaded 6001 articles of 13368
Loaded 6501 articles of 13368
Loaded 7001 articles of 13368
Loaded 8001 articles of 13368
Loaded 8501 articles of 13368
Loaded 9001 articles of 13368
Loaded 9501 articles of 13368
Loaded 10001 articles of 13368
Loaded 11001 articles of 13368
Loaded 11501 articles of 13368
Loaded 12001 articles of 13368
Loaded 12501 articles of 13368
Loaded 13001 articles of 13368


In [15]:
# keep_articles = []
# for i,(a_id, a_len, chunk_sents) in enumerate(required_items['test']):
# #     if i<=(244139-1):
# #         continue
# #     else:
#         if len(keep_articles)==60000:
#             break
#         if a_len!=10:
#             continue
#         keep_articles.append(a_id)
#         article_res = []
#         for (chunk_num, s_idx) in chunk_sents:
#             if loaded_current_chunk==chunk_num:
#                 s_emb, sent_embs_chunk = get_article_sent_emb(
#                     'test', chunk_num, s_idx, sent_embs_chunk
#                 )
#             else:
#                 print('  Loading chunk:', chunk_num)
#                 loaded_current_chunk = chunk_num
#                 s_emb, sent_embs_chunk = get_article_sent_emb(
#                     'test', chunk_num, s_idx, None
#                 )
#             article_res.append(s_emb)
#         res.append(article_res)
#         if i%500==0:
#             print('Loaded',i+1,'articles of',len(required_items['test']))

  Loading chunk: 1
Loaded 1 articles of 11490
Loaded 501 articles of 11490
Loaded 1001 articles of 11490
Loaded 1501 articles of 11490
Loaded 2001 articles of 11490
Loaded 2501 articles of 11490
Loaded 3001 articles of 11490
Loaded 3501 articles of 11490
Loaded 4001 articles of 11490
Loaded 4501 articles of 11490
Loaded 5001 articles of 11490
Loaded 5501 articles of 11490
Loaded 6001 articles of 11490
Loaded 6501 articles of 11490
Loaded 7001 articles of 11490
Loaded 7501 articles of 11490
Loaded 8001 articles of 11490
Loaded 8501 articles of 11490
Loaded 9001 articles of 11490
Loaded 9501 articles of 11490
Loaded 10001 articles of 11490
Loaded 10501 articles of 11490
Loaded 11001 articles of 11490


In [16]:
len(res), i, keep_articles[-1]

(11251, 11489, 'fffd506034c5275fe57220e669ad7e01605d597c')

In [21]:
# import pickle

# part = 1
# with open(
#     top_sentence_emb_path+'/validation/'+'validation_data_part_'+str(part)+'.pkl', 
#     'wb') as pickle_file:
#     pickle.dump(res, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)
# with open(
#     top_sentence_emb_path+'/validation/'+'validation_data_article_ids_part_'+str(part)+'.pkl', 
#     'wb') as pickle_file:
#     pickle.dump(keep_articles, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)

In [17]:
# import pickle

# part = 1
# with open(
#     top_sentence_emb_path+'/test/'+'test_data_part_'+str(part)+'.pkl', 
#     'wb') as pickle_file:
#     pickle.dump(res, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)
# with open(
#     top_sentence_emb_path+'/test/'+'test_data_article_ids_part_'+str(part)+'.pkl', 
#     'wb') as pickle_file:
#     pickle.dump(keep_articles, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)

In [15]:
import pickle

part = 1
with open(
    top_sentence_emb_path+'/train/'+'train_data_part_'+str(part)+'.pkl', 
    'wb') as pickle_file:
    pickle.dump(res, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)
with open(
    top_sentence_emb_path+'/train/'+'train_data_article_ids_part_'+str(part)+'.pkl', 
    'wb') as pickle_file:
    pickle.dump(keep_articles, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)

In [24]:
part = 2
with open(
    top_sentence_emb_path+'/train/'+'train_data_part_'+str(part)+'.pkl', 
    'wb') as pickle_file:
    pickle.dump(res, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)
with open(
    top_sentence_emb_path+'/train/'+'train_data_article_ids_part_'+str(part)+'.pkl', 
    'wb') as pickle_file:
    pickle.dump(keep_articles, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)

In [28]:
part = 3
with open(
    top_sentence_emb_path+'/train/'+'train_data_part_'+str(part)+'.pkl', 
    'wb') as pickle_file:
    pickle.dump(res, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)
with open(
    top_sentence_emb_path+'/train/'+'train_data_article_ids_part_'+str(part)+'.pkl', 
    'wb') as pickle_file:
    pickle.dump(keep_articles, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)

In [32]:
part = 4
with open(
    top_sentence_emb_path+'/train/'+'train_data_part_'+str(part)+'.pkl', 
    'wb') as pickle_file:
    pickle.dump(res, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)
with open(
    top_sentence_emb_path+'/train/'+'train_data_article_ids_part_'+str(part)+'.pkl', 
    'wb') as pickle_file:
    pickle.dump(keep_articles, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)

In [36]:
part = 5
with open(
    top_sentence_emb_path+'/train/'+'train_data_part_'+str(part)+'.pkl', 
    'wb') as pickle_file:
    pickle.dump(res, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)
with open(
    top_sentence_emb_path+'/train/'+'train_data_article_ids_part_'+str(part)+'.pkl', 
    'wb') as pickle_file:
    pickle.dump(keep_articles, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)